In [6]:
import os
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re

class RealCSVRAGEvaluator:
    def __init__(self, csv_path: str, sample_size: int = 5):
        """
        Gerçek CSV dosyanızı okuyarak RAG değerlendirmesi yapar
        """
        self.csv_path = csv_path
        self.sample_size = sample_size
        self.df = None
        self.vectorizer = TfidfVectorizer(stop_words='english')
    
    def load_real_csv(self):
        """Gerçek CSV dosyanızı okur ve işler"""
        print(f"📂 Gerçek CSV okunuyor: {self.csv_path}")
        
        if not os.path.exists(self.csv_path):
            raise FileNotFoundError(f"CSV dosyası bulunamadı: {self.csv_path}")
        
        # CSV'yi oku
        self.df = pd.read_csv(self.csv_path)
        print(f"✅ CSV yüklendi: {len(self.df)} satır")
        
        # Sütunları kontrol et ve yazdır
        print(f"📋 CSV Sütunları: {list(self.df.columns)}")
        print(f"📊 İlk 3 satır örneği:")
        print(self.df.head(3))
        
        # Gerekli sütunları hazırla
        self.prepare_evaluation_columns()
        
        # Sample al (büyük dosyalar için)
        if self.sample_size and len(self.df) > self.sample_size:
            self.df = self.df.sample(self.sample_size, random_state=42)
            print(f"🔽 Sample alındı: {len(self.df)} satır")
    
    def prepare_evaluation_columns(self):
        """RAG değerlendirmesi için gerekli sütunları hazırlar"""
        
        # Context sütunu oluştur (mevcut bilgilerden)
        if 'contexts' not in self.df.columns:
            context_parts = []
            if 'daire' in self.df.columns:
                context_parts.append(f"Daire: {self.df['daire']}")
            if 'mahkeme' in self.df.columns:
                context_parts.append(f"Mahkeme: {self.df['mahkeme']}")
            if 'karar_turu' in self.df.columns:
                context_parts.append(f"Karar Türü: {self.df['karar_turu']}")
            
            self.df["contexts"] = self.df.apply(
                lambda row: [
                    f"Daire: {row.get('daire', 'Bilinmeyen')}" if 'daire' in self.df.columns else "",
                    f"Mahkeme: {row.get('mahkeme', 'Bilinmeyen')}" if 'mahkeme' in self.df.columns else "",
                    f"Karar Türü: {row.get('karar_turu', 'Bilinmeyen')}" if 'karar_turu' in self.df.columns else ""
                ],
                axis=1
            )
        
        # Question sütunu oluştur
        if 'question' not in self.df.columns:
            if 'chunk_text' in self.df.columns:
                # chunk_text'den soru oluştur
                self.df['question'] = self.df['chunk_text'].apply(
                    lambda text: f"Bu hukuki karar hakkında açıklama yapınız: {text[:100]}..."
                )
            else:
                # Genel sorular oluştur
                self.df['question'] = "Bu hukuki karar hakkında bilgi veriniz."
        
        # Answer sütunu (chunk_text'i kullan)
        if 'answer' not in self.df.columns:
            if 'chunk_text' in self.df.columns:
                self.df['answer'] = self.df['chunk_text']
            else:
                raise ValueError("'chunk_text' sütunu bulunamadı!")
        
        # Ground truth sütunu
        if 'ground_truth' not in self.df.columns:
            if 'esasNo' in self.df.columns:
                self.df['ground_truth'] = self.df['esasNo'].astype(str)
            elif 'kararNo' in self.df.columns:
                self.df['ground_truth'] = self.df['kararNo'].astype(str)
            else:
                # Fallback: index kullan
                self.df['ground_truth'] = self.df.index.astype(str)
    
    def evaluate_faithfulness(self, answer: str, contexts: list) -> float:
        """Cevabın context'e ne kadar sadık olduğunu ölçer"""
        if not contexts or not answer:
            return 0.0
        
        # Context'leri birleştir ve temizle
        combined_context = ' '.join([ctx for ctx in contexts if ctx.strip()])
        
        if not combined_context.strip():
            return 0.0
        
        try:
            # TF-IDF ile benzerlik hesapla
            tfidf_matrix = self.vectorizer.fit_transform([answer, combined_context])
            similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]
            return min(similarity, 1.0)
        except:
            # Fallback: Kelime örtüşmesi
            answer_words = set(answer.lower().split())
            context_words = set(combined_context.lower().split())
            if not answer_words:
                return 0.0
            overlap = len(answer_words.intersection(context_words))
            return overlap / len(answer_words)
    
    def evaluate_relevancy(self, question: str, answer: str) -> float:
        """Cevabın soruyla ne kadar ilgili olduğunu ölçer"""
        if not question or not answer:
            return 0.0
        
        try:
            tfidf_matrix = self.vectorizer.fit_transform([question, answer])
            similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]
            return min(similarity, 1.0)
        except:
            # Fallback: Kelime örtüşmesi
            q_words = set(question.lower().split())
            a_words = set(answer.lower().split())
            if not q_words:
                return 0.0
            overlap = len(q_words.intersection(a_words))
            return overlap / len(q_words)
    
    def evaluate_context_precision(self, contexts: list, ground_truth: str) -> float:
        """Context'in ne kadar kesin/ilgili olduğunu ölçer"""
        if not contexts or not ground_truth:
            return 0.0
        
        relevant_contexts = 0
        valid_contexts = [ctx for ctx in contexts if ctx.strip()]
        
        if not valid_contexts:
            return 0.0
        
        for context in valid_contexts:
            if ground_truth.lower() in context.lower():
                relevant_contexts += 1
            elif any(word in context.lower() for word in ground_truth.lower().split() if len(word) > 2):
                relevant_contexts += 0.5
        
        return min(relevant_contexts / len(valid_contexts), 1.0)
    
    def evaluate_context_recall(self, contexts: list, ground_truth: str) -> float:
        """Ground truth'un context'te ne kadar coverage'ı var"""
        if not contexts or not ground_truth:
            return 0.0
        
        combined_context = ' '.join([ctx for ctx in contexts if ctx.strip()]).lower()
        gt_words = set(ground_truth.lower().split())
        
        if not gt_words or not combined_context:
            return 0.0
        
        found_words = sum(1 for word in gt_words if len(word) > 2 and word in combined_context)
        return found_words / len(gt_words) if gt_words else 0.0
    
    def evaluate_dataset(self) -> dict:
        """Dataset'teki tüm örnekleri değerlendirir"""
        if self.df is None:
            raise ValueError("CSV yüklenmemiş! Önce load_real_csv() çağırın.")
        
        results = {
            'faithfulness': [],
            'answer_relevancy': [],
            'context_precision': [],
            'context_recall': []
        }
        
        print(f"\n🔍 {len(self.df)} örnek değerlendiriliyor...")
        
        for idx, row in self.df.iterrows():
            print(f"⏳ İşleniyor: {idx+1}/{len(self.df)} - ID: {row.get('esasNo', idx)}")
            
            question = str(row['question'])
            answer = str(row['answer'])
            contexts = row['contexts'] if isinstance(row['contexts'], list) else [str(row['contexts'])]
            ground_truth = str(row['ground_truth'])
            
            # Debug bilgileri
            if idx == 0:
                print(f"📝 Örnek veri:")
                print(f"   Soru: {question[:100]}...")
                print(f"   Cevap: {answer[:100]}...")
                print(f"   Context: {contexts[:2]}")
                print(f"   Ground Truth: {ground_truth}")
            
            # Metrikleri hesapla
            faithfulness = self.evaluate_faithfulness(answer, contexts)
            relevancy = self.evaluate_relevancy(question, answer)
            precision = self.evaluate_context_precision(contexts, ground_truth)
            recall = self.evaluate_context_recall(contexts, ground_truth)
            
            results['faithfulness'].append(faithfulness)
            results['answer_relevancy'].append(relevancy)
            results['context_precision'].append(precision)
            results['context_recall'].append(recall)
        
        # Ortalama skorları hesapla
        avg_results = {
            metric: np.mean(scores) for metric, scores in results.items()
        }
        
        return avg_results, results


def main():
    print("🚀 Gerçek CSV ile RAG Değerlendirmesi")
    print("="*60)
    
    # Gerçek CSV dosya yolu (sizinkini kullanın)
    CSV_FILE = "/home/yapayzeka/ahsen_bulbul/model/langchain/recursive/yargitay_chunks.csv"
    
    try:
        # Evaluator'ü başlat
        evaluator = RealCSVRAGEvaluator(CSV_FILE, sample_size=5)  # 5 örnek test
        
        # Gerçek CSV'yi yükle
        evaluator.load_real_csv()
        
        # Değerlendirmeyi çalıştır
        avg_results, detailed_results = evaluator.evaluate_dataset()
        
        # Sonuçları göster
        print("\n" + "="*60)
        print("📈 GERÇEK CSV DEĞERLENDİRME SONUÇLARI")
        print("="*60)
        
        for metric, score in avg_results.items():
            print(f"{metric:20}: {score:.4f}")
        
        # En iyi ve en kötü örnekleri göster
        print(f"\n📊 En yüksek faithfulness skoru: {max(detailed_results['faithfulness']):.4f}")
        print(f"📊 En düşük faithfulness skoru: {min(detailed_results['faithfulness']):.4f}")
        
        # Sonuçları kaydet
        results_df = pd.DataFrame(detailed_results)
        output_file = "/tmp/real_csv_rag_evaluation.csv"
        results_df.to_csv(output_file, index=False)
        print(f"\n💾 Detaylı sonuçlar kaydedildi: {output_file}")
        
        return avg_results
        
    except FileNotFoundError as e:
        print(f"❌ Dosya hatası: {e}")
        print("CSV dosya yolunu kontrol edin!")
        return None
    except Exception as e:
        print(f"❌ Değerlendirme hatası: {e}")
        return None

if __name__ == "__main__":
    results = main()

🚀 Gerçek CSV ile RAG Değerlendirmesi
📂 Gerçek CSV okunuyor: /home/yapayzeka/ahsen_bulbul/model/langchain/recursive/yargitay_chunks.csv
✅ CSV yüklendi: 137 satır
📋 CSV Sütunları: ['_id', 'location', 'esasNo', 'kararNo', 'extractedDates', 'esasNo_num', 'esasNo_tip', 'kararNo_num', 'kararNo_tip', 'daire', 'mahkeme', 'karar_turu', 'chunk_id', 'chunk_index', 'total_chunks', 'chunk_text', 'chunk_length']
📊 İlk 3 satır örneği:
                        _id        location        esasNo      kararNo  \
0  6750524485b6640290c37b06  6.HukukDairesi   2023/596 E.  2024/257 K.   
1  6750524485b6640290c37b07  6.HukukDairesi  2022/3281 E.  2024/117 K.   
2  6750524485b6640290c37b07  6.HukukDairesi  2022/3281 E.  2024/117 K.   

                                      extractedDates esasNo_num esasNo_tip  \
0                                         2024-01-18   2023/596          E   
1  2009-06-26,2009-12-25,2009-12-31,2010-01-04,20...  2022/3281          E   
2  2009-06-26,2009-12-25,2009-12-31,2010-01-0

In [1]:
import pandas as pd
import random

def generate_qa_dataset(csv_path, output_path, sample_size=None, seed=42):
    """
    CSV'den otomatik Q&A dataset üretir.
    
    Args:
        csv_path (str): Chunk CSV dosyası
        output_path (str): Oluşturulacak CSV dosyası
        sample_size (int, optional): Kaç satır örnek alınacak. Default tüm veri.
        seed (int): Random seed
    """
    df = pd.read_csv(csv_path)
    print(f"Toplam satır: {len(df)}")

    # Sample al
    if sample_size and len(df) > sample_size:
        df = df.sample(sample_size, random_state=seed)
        print(f"Sample alındı: {len(df)} satır")

    # Soru oluşturma (basit template)
    def create_question(row):
        # Örnek soru şablonları
        templates = [
            f"{row['daire']} tarafından verilen {row['karar_turu']} kararı hakkında bilgi veriniz.",
            f"Esas No {row['esasNo']} ve Karar No {row['kararNo']} ile ilgili kararda ne söyleniyor?",
            f"{row['mahkeme']} kararına göre durum nedir?",
            f"{row['daire']} {row['karar_turu']} kararında ne karar verilmiş?"
        ]
        return random.choice(templates)

    df['question'] = df.apply(create_question, axis=1)
    df['answer'] = df['chunk_text']
    df['context'] = df['chunk_text']  # RAG için context olarak chunk'ı kullanabiliriz

    # Sadece gerekli sütunları seç
    qa_df = df[['question', 'answer', 'context', 'esasNo', 'kararNo', 'daire', 'mahkeme', 'karar_turu']]

    # Kaydet
    qa_df.to_csv(output_path, index=False)
    print(f"Q&A dataset oluşturuldu ve kaydedildi: {output_path}")

    return qa_df


# Kullanım örneği
csv_file = "/home/yapayzeka/ahsen_bulbul/model/langchain/recursive/yargitay_chunks.csv"
output_file = "/home/yapayzeka/ahsen_bulbul/model/langchain/recursive/yargitay_qa_dataset.csv"
qa_df = generate_qa_dataset(csv_file, output_file, sample_size=50)
print(qa_df.head())


Toplam satır: 137
Sample alındı: 50 satır
Q&A dataset oluşturuldu ve kaydedildi: /home/yapayzeka/ahsen_bulbul/model/langchain/recursive/yargitay_qa_dataset.csv
                                              question  \
105  6. Hukuk Dairesi tarafından verilen RED kararı...   
104  6. Hukuk Dairesi tarafından verilen RED kararı...   
12   Asliye Hukuk Mahkemesi Taraflar arası kararına...   
26   Esas No 2023/4229 E. ve Karar No 2024/136 K. i...   
123  [...] Bölge Adliye Mahkemesi 46. Hukuk Dairesi...   

                                                answer  \
105  6. Hukuk Dairesi 2023/4278 E. , 2024/689 K. \n...   
104  . Davalı vekili 19.12.2023 tarihli dilekçesi i...   
12   . 2. İlgili Hukuk 6100 sayılı Hukuk Muhakemele...   
26   . B. İstinaf Sebepleri Alacaklı [...] [...] Tı...   
123  . III. İLK DERECE MAHKEMESİ KARARI İlk Derece ...   

                                               context        esasNo  \
105  6. Hukuk Dairesi 2023/4278 E. , 2024/689 K. \n...  2023/4278 E.  

In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams, PointStruct
from qdrant_client.http import models
import uuid
from tqdm import tqdm
import json
from typing import List, Dict, Any
import re
from dotenv import load_dotenv
import os
import pandas as pd

/home/yapayzeka/ahsen_bulbul/ahsen/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class LegalDocumentVectorDB:
    def __init__(self, qdrant_url: str, api_key: str, collection_name: str = "hukuki kararlar"):
        """
        Hukuki belgeler için vector database sınıfı
        
        Args:
            qdrant_url: Qdrant sunucu URL'i
            api_key: Qdrant API anahtarı  
            collection_name: Collection adı
        """
        self.client = QdrantClient(
            url=qdrant_url,
            api_key=api_key,
            timeout=60
        )
        self.collection_name = collection_name
        
        # Türkçe için optimize edilmiş multilingual model
        print("Sentence Transformer modeli yükleniyor...")
        self.model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
        print("Model başarıyla yüklendi!")
        
        # Vector boyutunu al
        self.vector_size = self.model.get_sentence_embedding_dimension()
        print(f"Vector boyutu: {self.vector_size}")

    def create_collection(self, recreate: bool = False):
        """Collection oluştur"""
        try:
            if recreate:
                self.client.delete_collection(collection_name=self.collection_name)
                print(f"Eski collection '{self.collection_name}' silindi.")
        except:
            pass
        
        try:
            self.client.create_collection(
                collection_name=self.collection_name,
                vectors_config=VectorParams(
                    size=self.vector_size, 
                    distance=Distance.COSINE
                )
            )
            print(f"Collection '{self.collection_name}' oluşturuldu.")
        except Exception as e:
            if "already exists" in str(e).lower():
                print(f"Collection '{self.collection_name}' zaten mevcut.")
            else:
                raise e

    def clean_text(self, text: str) -> str:
        """Metni temizle"""
        if pd.isna(text):
            return ""
        
        # Encoding sorunlarını düzelt
        text = str(text)
        replacements = {
            'Ã¤': 'ä', 'Ã¶': 'ö', 'Ã¼': 'ü', 'ÃŸ': 'ß',
            'Ã‡': 'Ç', 'Ä±': 'ı', 'Ä°': 'İ', 'ÅŸ': 'ş',
            'Ä\x9f': 'ğ', 'Ã§': 'ç', 'Ã¶': 'ö', 'Ã¼': 'ü'
        }
        
        for old, new in replacements.items():
            text = text.replace(old, new)
        
        # Fazla boşlukları temizle
        text = re.sub(r'\s+', ' ', text)
        text = text.strip()
        
        return text

    def process_csv(self, csv_path: str) -> pd.DataFrame:
        """CSV dosyasını işle"""
        print(f"CSV dosyası okunuyor: {csv_path}")
        
        # Dosya varlığını kontrol et
        import os
        if not os.path.exists(csv_path):
            print(f"❌ HATA: Dosya bulunamadı: {csv_path}")
            return None
        
        try:
            # Encoding denemesi
            try:
                df = pd.read_csv(csv_path, encoding='utf-8')
                print("✅ UTF-8 encoding ile başarıyla okundu")
            except UnicodeDecodeError:
                df = pd.read_csv(csv_path, encoding='latin-1')
                print("✅ Latin-1 encoding ile başarıyla okundu")
        except Exception as e:
            print(f"❌ CSV okuma hatası: {e}")
            return None
        
        print(f"Toplam satır sayısı: {len(df)}")
        print(f"Sütunlar: {df.columns.tolist()}")
        
        # Boş chunk_text'leri filtrele
        initial_count = len(df)
        df = df.dropna(subset=['chunk_text'])
        df = df[df['chunk_text'].str.strip() != '']
        final_count = len(df)
        
        print(f"Boş metin filtrelemesi: {initial_count} -> {final_count}")
        
        # Metinleri temizle
        df['chunk_text_clean'] = df['chunk_text'].apply(self.clean_text)
        
        return df

    def create_embeddings_batch(self, texts: List[str], batch_size: int = 32) -> List[List[float]]:
        """Metinleri batch halinde embedding'e çevir"""
        embeddings = []
        
        for i in tqdm(range(0, len(texts), batch_size), desc="Embedding oluşturuluyor"):
            batch = texts[i:i+batch_size]
            batch_embeddings = self.model.encode(
                batch, 
                convert_to_numpy=True,
                show_progress_bar=False,
                normalize_embeddings=True  # Cosine similarity için normalize et
            )
            embeddings.extend(batch_embeddings.tolist())
            print(embeddings)
        
        return embeddings

    def upload_to_qdrant(self, df: pd.DataFrame, batch_size: int = 100):
        """DataFrame'i Qdrant'a yükle"""
        print("Qdrant'a yükleme başlıyor...")
        
        # Embeddings oluştur
        texts = df['chunk_text_clean'].tolist()
        embeddings = self.create_embeddings_batch(texts, batch_size=32)
        
        # Points oluştur
        points = []
        for idx, (_, row) in enumerate(df.iterrows()):
            payload = {
                "document_id": str(row['_id']),
                "location": str(row['location']),
                "esas_no": str(row['esasNo']),
                "karar_no": str(row['kararNo']),
                "dates": str(row['extractedDates']),
                "daire": str(row['daire']),
                "mahkeme": str(row['mahkeme']),
                "karar_turu": str(row['karar_turu']),
                "chunk_id": str(row['chunk_id']),
                "chunk_index": int(row['chunk_index']),
                "total_chunks": int(row['total_chunks']),
                "chunk_text": str(row['chunk_text_clean']),
                "chunk_length": int(row['chunk_length'])
            }
            
            point = PointStruct(
                id=str(uuid.uuid4()),
                vector=embeddings[idx],
                payload=payload
            )
            points.append(point)
        
        # Batch halinde yükle
        for i in tqdm(range(0, len(points), batch_size), desc="Qdrant'a yükleniyor"):
            batch_points = points[i:i+batch_size]
            
            try:
                self.client.upsert(
                    collection_name=self.collection_name,
                    points=batch_points
                )
            except Exception as e:
                print(f"Yükleme hatası (batch {i//batch_size + 1}): {e}")
                continue
        
        print(f"Toplam {len(points)} doküman başarıyla yüklendi!")

    def search(self, query: str, limit: int = 5, score_threshold: float = 0.5) -> List[Dict[str, Any]]:
        """Semantik arama yap"""
        print(f"Arama yapılıyor: '{query}'")
        
        # Query embedding'i oluştur
        query_embedding = self.model.encode(
            [query], 
            convert_to_numpy=True,
            normalize_embeddings=True
        )[0].tolist()
        
        # Arama yap (query_points kullan)
        search_results = self.client.query_points(
            collection_name=self.collection_name,
            query=query_embedding,
            limit=limit,
            score_threshold=score_threshold
        )
        
        results = []
        for result in search_results.points:
            results.append({
                "score": result.score,
                "payload": result.payload
            })
        
        return results

    def advanced_search(self, query: str, filters: Dict = None, limit: int = 5) -> List[Dict[str, Any]]:
        """Gelişmiş filtreleme ile arama"""
        query_embedding = self.model.encode(
            [query], 
            convert_to_numpy=True,
            normalize_embeddings=True
        )[0].tolist()
        
        # Filter oluştur
        filter_conditions = None
        if filters:
            conditions = []
            
            if 'daire' in filters:
                conditions.append(models.FieldCondition(
                    key="daire",
                    match=models.MatchValue(value=filters['daire'])
                ))
            
            if 'karar_turu' in filters:
                conditions.append(models.FieldCondition(
                    key="karar_turu", 
                    match=models.MatchValue(value=filters['karar_turu'])
                ))
            
            if 'year' in filters:
                conditions.append(models.FieldCondition(
                    key="dates",
                    match=models.MatchText(text=str(filters['year']))
                ))
            
            if conditions:
                filter_conditions = models.Filter(must=conditions)
        
        # query_points kullan
        search_results = self.client.query_points(
            collection_name=self.collection_name,
            query=query_embedding,
            query_filter=filter_conditions,
            limit=limit
        )
        
        results = []
        for result in search_results.points:
            results.append({
                "score": result.score,
                "payload": result.payload
            })
        
        return results

    def get_collection_info(self):
        """Collection bilgilerini getir"""
        try:
            info = self.client.get_collection(collection_name=self.collection_name)
            result = {
                "status": str(info.status),
                "vectors_count": info.vectors_count if hasattr(info, 'vectors_count') else 0,
            }
            
            # Mevcut attributeları kontrol et ve ekle
            if hasattr(info, 'segments_count'):
                result["segments_count"] = info.segments_count
            if hasattr(info, 'indexed_vectors_count'):
                result["indexed_vectors_count"] = info.indexed_vectors_count
            if hasattr(info, 'points_count'):
                result["points_count"] = info.points_count
                
            return result
        except Exception as e:
            return {"error": str(e)}

NameError: name 'pd' is not defined

In [1]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_recall, context_precision

# Qdrant tarafı (LegalDocumentVectorDB daha önce tanımladığın class)
# from your_module import LegalDocumentVectorDB

# ENV yükleme

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
qdrant_url = os.getenv("QDRANT_URL")
api_key = os.getenv("QDRANT_API_KEY")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY bulunamadı. qdrant.env dosyasını kontrol et.")

CSV_FILE = "/home/yapayzeka/ahsen_bulbul/model/langchain/recursive/yargitay_chunks.csv"

# ---------------- RAG Evaluator ---------------- #
class RAGEvaluator:
    def __init__(self, csv_path: str):
        self.csv_path = csv_path
        self.df = None
        self.dataset = None

    def load_csv(self):
        print(f"📂 CSV okunuyor: {self.csv_path}")
        self.df = pd.read_csv(self.csv_path)
        print(f"✅ Toplam satır: {len(self.df)}")
        self.df["contexts"] = self.df.apply(
            lambda row: [
                f"Daire: {row['daire']}",
                f"Mahkeme: {row['mahkeme']}",
                f"Karar Türü: {row['karar_turu']}"
            ],
            axis=1
        )
        self.df["question"] = self.df["chunk_text"]
        self.df["ground_truth"] = self.df["esasNo"].astype(str)

    def prepare_dataset(self):
        if self.df is None:
            raise ValueError("CSV yüklenmemiş. Önce load_csv() çağır.")
        self.dataset = Dataset.from_pandas(
            self.df[["question", "chunk_text", "contexts", "ground_truth"]].rename(
                columns={"chunk_text": "answer"}
            )
        )
        print("📊 Dataset hazır.")

    def evaluate_rag(self, output_csv: str = "/tmp/rag_evaluation_results.csv"):
        if self.dataset is None:
            raise ValueError("Dataset hazırlanmadı. Önce prepare_dataset() çağır.")
        print("🔍 RAG değerlendirmesi başlatılıyor...")
        results = evaluate(
            self.dataset,
            metrics=[faithfulness, answer_relevancy, context_precision, context_recall]
        )
        # EvaluationResult nesnesinden dict çıkar
        scores = {metric: results[metric] for metric in ["faithfulness", "answer_relevancy", "context_precision", "context_recall"]}
        print("\n============================================================")
        print("📈 RAG DEĞERLENDİRME SONUÇLARI")
        print("============================================================")
        for metric, score in scores.items():
            print(f"{metric:20} : {score:.4f}")
        print("============================================================")
        # Detaylı sonuçları CSV’ye kaydet
        self.df.to_csv(output_csv, index=False)
        print(f"💾 Detaylı sonuçlar kaydedildi: {output_csv}")
        return scores

# ---------------- Main ---------------- #
def main():
    # Qdrant veri tabanı
    db = LegalDocumentVectorDB(
        qdrant_url=qdrant_url,
        api_key=api_key,
        collection_name="hukuki_kararlar"
    )
    db.create_collection(recreate=True)
    df = pd.read_csv(CSV_FILE)
    db.upload_to_qdrant(df, batch_size=50)

    # Collection bilgilerini göster
    info = db.get_collection_info()
    print("\n=== Qdrant Collection Bilgileri ===")
    print(json.dumps(info, indent=2, ensure_ascii=False))

    # Örnek arama
    print("\n=== Örnek Aramalar ===")
    results = db.search("ihtiyati tedbir tazminat", limit=3, score_threshold=0.6)
    for i, result in enumerate(results, 1):
        print(f"\n{i}. Sonuç (Skor: {result['score']:.3f})")
        print(f"   Daire: {result['payload']['daire']}")
        print(f"   Esas No: {result['payload']['esas_no']}")
        print(f"   Karar No: {result['payload']['karar_no']}")
        print(f"   Metin: {result['payload']['chunk_text'][:200]}...")

    # RAG Değerlendirmesi
    evaluator = RAGEvaluator(CSV_FILE)
    evaluator.load_csv()
    evaluator.prepare_dataset()
    evaluator.evaluate_rag(output_csv="/tmp/real_csv_rag_evaluation.csv")

if __name__ == "__main__":
    main()


/home/yapayzeka/ahsen_bulbul/ahsen/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'LegalDocumentVectorDB' is not defined